ソース
https://huggingface.co/aken12/splade-japanese-v3

In [3]:
%pip install fugashi ipadic unidic-lite

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import torch.nn.functional as F

class SparseEncoder:
    def __init__(self, model_id="aken12/splade-japanese-v3"):
        self._tokenizer = AutoTokenizer.from_pretrained(model_id)
        self._model = AutoModelForMaskedLM.from_pretrained(model_id)
        self._model.eval()
        # extract the ID position to text token mappings
        self._idx2token = {
            idx: token for token, idx in self._tokenizer.get_vocab().items()
            }

    def encode(self, text):
        tokens = self._tokenizer([text], return_tensors='pt')
        with torch.no_grad():
            output = self._model(**tokens)

        vec = torch.max(
            torch.log(1 + torch.relu(output.logits)) * tokens.attention_mask.unsqueeze(-1),
            dim=1)[0].squeeze()

        # extract non-zero positions
        cols = vec.nonzero().squeeze().cpu().tolist()
        # extract the non-zero values
        weights = vec[cols].cpu().tolist()
        # use to create a dictionary of token ID to weight
        sparse_dict = dict(zip(cols, weights))
        # map token IDs to human-readable tokens
        sparse_dict_tokens = {
            self._idx2token[idx]: round(weight, 4) for idx, weight in zip(cols, weights)
        }
        # sort so we can see most relevant tokens first
        sparse_dict_tokens = {
            k: v for k, v in sorted(
                sparse_dict_tokens.items(),
                key=lambda item: item[1],
                reverse=True
            )
        }
        return sparse_dict_tokens

In [9]:
sparce_encoder = SparseEncoder()
text =  "プログラム細胞死 (PCD) は、生物内の細胞の制御された死です。"
sparce_vector = sparce_encoder.encode(text)
sparce_vector

{'は': 2.3874,
 '。': 2.3745,
 'の': 2.3735,
 'に': 2.3711,
 'が': 2.3233,
 'で': 2.3156,
 '、': 2.3085,
 'を': 2.2732,
 '死': 2.2482,
 'プログラム': 2.2347,
 'と': 2.2329,
 'から': 2.2054,
 'も': 2.1424,
 '細胞': 2.1168,
 '##d': 2.0952,
 '・': 2.0932,
 '(': 2.0511,
 ')': 2.0483,
 'や': 2.045,
 '語': 1.9052,
 'プログラミング': 1.901,
 'へ': 1.9008,
 'など': 1.8752,
 ':': 1.8096,
 '用語': 1.8066,
 '制御': 1.7713,
 'な': 1.7551,
 'て': 1.7533,
 'p': 1.7059,
 '生物': 1.6927,
 '##ad': 1.5953,
 '病': 1.5534,
 'です': 1.5413,
 '##ド': 1.5405,
 'd': 1.5251,
 '」': 1.5072,
 'pp': 1.5048,
 'セル': 1.5016,
 'ph': 1.4964,
 'れ': 1.4715,
 '死亡': 1.4604,
 'コンピュータ': 1.4252,
 '##gram': 1.4239,
 '##c': 1.421,
 '致死': 1.4139,
 '商標': 1.3991,
 '##死': 1.3957,
 'ピー': 1.3275,
 '死体': 1.3248,
 '内': 1.3193,
 '死に': 1.3068,
 'コントロール': 1.3028,
 'た': 1.2939,
 '「': 1.2937,
 'さ': 1.2911,
 '生命': 1.2822,
 'にて': 1.2537,
 'し': 1.2445,
 'こと': 1.2401,
 'まで': 1.2156,
 'れる': 1.2155,
 '##ディー': 1.2151,
 '##ピュ': 1.2083,
 '##プト': 1.208,
 '##id': 1.2064,
 'ピ': 1.2047,
 '##de': 1

In [10]:
# 2つのベクトルのスコア計算（類似単語が多いとスコアが高くなる）
def get_score(vec1, vec2):
  score = 0.0
  for k,v in vec1.items():
    if k in vec2:
      score += v * vec2.get(k)
  return score

# 検索対象の文書的なもの
texts = [
    "グローバル市場の動向は、再生可能エネルギー源へのシフトを示しています。",
    "革新的な技術がヘルスケア産業を変革しています。",
    "教育改革は将来の労働力開発に不可欠です。",
    "気候変動と戦うために、環境保全の努力が増加しています。",
    "人工知能はビジネスの運営方法を革命的に変えています。",
]

sparce_encoder = SparseEncoder()
sparse_vectors = [sparce_encoder.encode(t) for t in texts]

# 検索語的なもの
target = "気候行動"
target_vector = sparce_encoder.encode(target)

sorted_scores = sorted(zip(texts, sparse_vectors), key=lambda x: get_score(x[1], target_vector), reverse=True)
for text, vec in sorted_scores:
  score = get_score(vec, target_vector)
  print(f"{score} {text}")

92.77301443 気候変動と戦うために、環境保全の努力が増加しています。
86.82133408000003 グローバル市場の動向は、再生可能エネルギー源へのシフトを示しています。
85.35484702999999 人工知能はビジネスの運営方法を革命的に変えています。
83.19003284000001 革新的な技術がヘルスケア産業を変革しています。
82.58787439 教育改革は将来の労働力開発に不可欠です。
